In [ ]:
#Imports
import sys #Python
import sklearn #Machine learning library
import numpy as np #numerical packages in python
import scipy as scp #Another numerical package, unused directly but is implicitly used in sklearn
import pandas as pd #Package for data manipulation and analysis
import matplotlib # plotting library
import os
import time
import random 

# SKlearn imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier